In [ ]:
# Standard libraries: https://docs.python.org/3/library/
import math
import numpy as np
# Analysis and plotting modules
import pandas as pd
import plotly

# radCAD modules
from radcad import Model, Simulation

import setup
import copy

In [ ]:
from model.stochastic_processes import create_stochastic_process_realizations
import experiments.simulation_configuration as simulation
from experiments.run import run

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

## Model

In [ ]:
monte_carlo_runs = 10

In [ ]:
# Create a simulation for each analysis
simulation = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
#model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
#simulation = Simulation(model=model, timesteps=timesteps, runs=monte_carlo_runs)

In [ ]:
# Experiment configuration
simulation.runs = monte_carlo_runs

# Experiment configuration
simulation.model.initial_state.update({})

# Override default experiment System Parameters
simulation.model.params.update({})

In [ ]:
df, exceptions = run(simulation)

## analysis, post processing

In [ ]:
df.query('run==1')

In [ ]:
df.plot(
    x='timestep',
    y=['volatile_asset_price']
)

In [ ]:
df.plot(
    x='timestep',
    y=['discounted_payoff']
)

In [ ]:
def get_option_payoff(df):
    
    discounted_payoffs = dict()

    for run in df["run"].unique():
        
        df_ = df.query('run == @run')

        d_payoff = df_['discounted_payoff'].iloc[-1]
        
        discounted_payoffs[run] = d_payoff
        
    return pd.DataFrame(discounted_payoffs.items(), columns=['run', 'option_payoff'])

In [ ]:
initial_price = simulation.model.initial_state['volatile_asset_price']

In [ ]:
payoff_df = get_option_payoff(df)
option_price = payoff_df['option_payoff'].mean()

In [ ]:
print('Option price (monte carlo) is:', option_price)

# ignore

In [ ]:
class European_Call_Payoff:

    def __init__(self, strike):
        self.strike = strike

    def get_payoff(self, stock_price):
        if stock_price > self.strike:
            return stock_price - self.strike
        else:
            return 0

In [ ]:
ec = European_Call_Payoff(initial_price)

In [ ]:
def get_option_payoff(option, df):
    
    discounted_payoffs = dict()

    for run in df["run"].unique():
        
        df_ = df.query('run == @run')

        va_price = df_['volatile_asset_price'].iloc[-1]
        risk_free_rate = df_['risk_free_rate'].iloc[-1]
        rf_daily = risk_free_rate / (timesteps * dt)
        d_payoff = option.get_payoff(va_price) * np.exp(-rf_daily* T)
        
        discounted_payoffs[run] = d_payoff
        
    return pd.DataFrame(discounted_payoffs.items(), columns=['run', 'option_payoff'])